In [1]:
import pandas as pd
import numpy as np
from glob import glob #filename pattern matching module
import os
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

#### 1. channel dataset

In [68]:
file_path = '.\data_source\daily_op_data.xlsx'
sheet_names = [f'{i}月个别渠道分析（官网）' for i in range(1, 12)]

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Rex\AppData\Local\Temp\ipykernel_10588\1820230023.py:1: SyntaxWarning: invalid escape sequence '\d'
  file_path = '.\data_source\daily_op_data.xlsx'


In [54]:
try:
    with pd.ExcelFile(file_path) as xls:
        dfs = [pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names]
    merged_df = pd.concat(dfs, ignore_index=True)
except Exception as e:
    print(f"讀取或合併 Excel 工作表時發生錯誤: {e}")
    raise

In [55]:
# merged_df = pd.DataFrame()
# for sheet in sheet_names:
#     df = pd.read_excel(file_path, sheet_name=sheet)
#     merged_df = pd.concat([merged_df, df], ignore_index=True)

In [56]:
org_cols = merged_df.columns.tolist()

In [57]:
org_cols

['统计日期',
 '渠道ID',
 '渠道名称',
 '单价',
 '活跃用户数',
 '新增注册用户数',
 '扣量后注册',
 '扣量％数',
 '昨日安装',
 '对比昨日',
 '昨日同比率',
 '扣量前费用',
 '扣量后费用',
 '渠道营收',
 '充值人数',
 '新增会员数',
 '总充值金额',
 '直推充值金额',
 '提现金额',
 '购买会员金额',
 '安卓购买会员金额',
 'ios购买会员金额',
 'h5购买会员金额',
 '直推购买会员金额',
 '邀请购买会员金额',
 '内容付费金额',
 '打赏收入',
 '游戏消耗金额',
 '邀请返利',
 '次日留存',
 '三日留存',
 '7日留存',
 'cpa单价',
 'cpa收益',
 'cps占比',
 'cps收益',
 '毛利率',
 '日活付费率',
 '新增付费率',
 '二次付费率',
 'arpu',
 'arppu']

In [58]:
english_columns = [
    'Date', 'Channel ID', 'Channel Name', 'Unit Price', 'Active Users', 'New Registrations', 'Post-Deduction Registrations',
    'Deduction Percentage', 'Installation Yesterday', 'Installation DoD', 'Installation DoD pct', 'Pre-Deduction Cost',
    'Post-Deduction Cost', 'Channel Revenue', 'Recharge Users', 'New Membership', 'Recharge Amount', 'Direct Referral Recharge Amount',
    'Withdrawal Amount', 'Membership Revenue', 'Android Membership Revenue', 'iOS Membership Revenue',
    'H5 Membership Revenue', 'Direct Referral Membership Revenue', 'Invited Membership Revenue', 'Content Fee Revenue',
    'Tip Revenue', 'Game Revenue', 'Invitation Rebate', '1d Retention', '3d Retention', '7d Retention',
    'CPA Unit Price', 'CPA Revenue', 'CPS Ratio', 'CPS Revenue', 'GPM', 'Paid Active Users Rate', 'Paid New Registration Rate',
    'Repurchase Rate', 'ARPU', 'ARPPU'
]


In [59]:
pair = dict(zip(org_cols, english_columns))

In [60]:
# 輸出欄位對照表
ch_df_trans = pd.DataFrame(list(pair.items()), columns=['中文欄位名', '英文欄位名'])
ch_df_trans.to_excel('ch_df_trans.xlsx', index=False)

In [95]:
ch_df_trans

,中文欄位名,英文欄位名
0,统计日期,Date
1,渠道ID,Channel ID
2,渠道名称,Channel Name
3,单价,Unit Price
4,活跃用户数,Active Users
5,新增注册用户数,New Registrations
6,扣量后注册,Post-Deduction Registrations
7,扣量％数,Deduction Percentage
8,昨日安装,Installation Yesterday
9,对比昨日,Installation DoD


In [61]:
merged_df.rename(columns=pair, inplace=True)

In [62]:
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.normalize()

In [63]:
zero_columns = merged_df.columns[(merged_df == 0).all()].tolist()

In [64]:
df = merged_df.drop(columns=['Channel Name'])

In [65]:
df = df.drop(columns=zero_columns)

In [66]:
# df.to_excel('sp001_channel_dataset.xlsx', index=False)

In [67]:
df.to_csv('sp001_tb_channel.csv', index=False, encoding='utf-8')

#### 2. op_data

In [133]:
file_path = '.\data_source\daily_op_data.xlsx'
sheet_names = ['11月总表', '10月总表', '9月总表', '8月总表',
                '7月总表', '6月总表', '5月总表', '4月总表', 
                '3月总表', '2月总表', '1月总表']

In [134]:
dfs = []
for sheet in sheet_names:
    dfs.append(pd.read_excel(file_path, sheet_name=sheet))
    # merged_df = pd.concat([merged_df, df], ignore_index=True)

In [135]:
# 1-8月都要修改欄位
# print(len(dfs[0].columns))
# print(len(dfs[1].columns))
# print(len(dfs[2].columns))
# print(len(dfs[3].columns))

In [136]:
# 更正11月含有\n的欄位
dfs[0].columns = dfs[0].columns.str.replace('\n', '', regex=False)
# 以11月欄位為基準
reference_columns = set(dfs[0].columns)
# 比對所有 DataFrame 的欄位名稱是否與11月欄位名稱相等
list(df.columns.equals(reference_columns) for df in dfs)

[False, False, False, False, False, False, False, False, False, False, False]

In [137]:
for index, df in enumerate(dfs):
    print(f"Index: {index}, DataFrame shape: {df.shape}")

Index: 0, DataFrame shape: (30, 49)
Index: 1, DataFrame shape: (31, 49)
Index: 2, DataFrame shape: (31, 49)
Index: 3, DataFrame shape: (32, 43)
Index: 4, DataFrame shape: (32, 43)
Index: 5, DataFrame shape: (31, 43)
Index: 6, DataFrame shape: (32, 43)
Index: 7, DataFrame shape: (32, 43)
Index: 8, DataFrame shape: (32, 43)
Index: 9, DataFrame shape: (29, 21)
Index: 10, DataFrame shape: (32, 21)


In [138]:
## 哪個 DataFrame 的欄位不相等，並找出多餘或缺少的欄位
for index, df in enumerate(dfs):
    current_columns = set(df.columns)
    if current_columns != reference_columns:
        missing_columns = reference_columns - current_columns
        extra_columns = current_columns - reference_columns
        display(f"與第{index+1}個df的欄位不相等")
        # display(f"預期欄位: {reference_columns}")
        # display(f"實際欄位: {current_columns}")
        if missing_columns:
            display(f"缺少的欄位: {missing_columns}")
            # for col in missing_columns:
            #     df[col] = pd.NA
            # dfs[index] = df
            # display(f"更新後的第 {index} 個 DataFrame 的欄位: {df.columns}")
            all_columns_equal_after_update = all(set(df.columns) == reference_columns for df in dfs)
            print(f"更新後所有 DataFrame 的欄位是否相等: {all_columns_equal_after_update}")
        if extra_columns:
            display(f"多出的欄位: {extra_columns}")

'與第2個df的欄位不相等'

"缺少的欄位: {'银行卡支付', '支付宝支付'}"

更新後所有 DataFrame 的欄位是否相等: False


"多出的欄位: {'支付宝支付\\n', '银行卡支付\\n'}"

'與第3個df的欄位不相等'

...

"多出的欄位: {'活跃用户数'}"

'與第11個df的欄位不相等'

"缺少的欄位: {'新增会员数同比', '备注', '成功订单数', '云闪付', '实际拉起订单数', '新用户充值人数', '游戏营收', '提现（元）', '留存同比', '登录同比', '安卓人均充值', 'H5人均充值', '会员总收益', '登录用户数', '支付宝支付', '游戏人均付费', '新用户充值', '新增注册同比', '充值人数同比', '安卓充值', '成功率', 'H5充值', '拉起成功订单数', '留存用户', '（旧版）苹果人均充值', '人工支付', '（旧版）苹果充值', '银行卡支付', '微信支付', 'QQ钱包'}"

更新後所有 DataFrame 的欄位是否相等: False


"多出的欄位: {'活跃用户数', '提现金额'}"

In [139]:
# 支付欄位順序
payment_col = list(['人工支付', '支付宝支付', '银行卡支付', '微信支付', '云闪付', 'QQ钱包'])
payment_col

['人工支付', '支付宝支付', '银行卡支付', '微信支付', '云闪付', 'QQ钱包']

In [140]:
#處理10月的欄位: '银行卡支付', '支付宝支付'
dfs[1].columns = dfs[1].columns.str.replace('\n', '', regex=False)

In [141]:
#處理9月的欄位: 

In [142]:
for index, col in enumerate(dfs[2].columns):
    print(f"order: {index}, col_name: {col}")

order: 0, col_name: 统计日期
order: 1, col_name: 登录用户数
order: 2, col_name: 登录同比
order: 3, col_name: 新增注册用户数
order: 4, col_name: 新增注册同比
order: 5, col_name: 扣量后注册
order: 6, col_name: 留存用户
order: 7, col_name: 留存同比
order: 8, col_name: 昨日安装
order: 9, col_name: 对比昨日
order: 10, col_name: 昨日同比率
order: 11, col_name: 扣量前费用
order: 12, col_name: 扣量后费用
order: 13, col_name: 渠道营收
order: 14, col_name: 提现（元）
order: 15, col_name: 充值人数
order: 16, col_name: 充值人数同比
order: 17, col_name: 新用户充值
order: 18, col_name: 新用户充值人数
order: 19, col_name: 新增会员数
order: 20, col_name: 新增会员数同比
order: 21, col_name: 总充值金额
order: 22, col_name: 人工支付充值
order: 23, col_name: 支付宝充值
order: 24, col_name: 银联充值
order: 25, col_name: 微信支付
order: 26, col_name: 云闪付充值
order: 27, col_name: QQ钱包      充值
order: 28, col_name: 直推充值金额
order: 29, col_name: （旧版）苹果充值
order: 30, col_name: （旧版）苹果人均充值
order: 31, col_name: 安卓充值
order: 32, col_name: 安卓人均充值
order: 33, col_name: H5充值
order: 34, col_name: H5人均充值
order: 35, col_name: 实际拉起订单数
order: 36, col_name: 

In [143]:
# 替換正確支付欄位名稱
dfs[2].columns.values[22:28] = payment_col

In [144]:
# # 9月 舊欄位值-> 新欄位
# old_new_columns = {
#     'QQ钱包      充值': 'QQ钱包',
#     '支付宝充值': '支付宝支付',
#     '云闪付充值': '云闪付',
#     '银联充值': '银行卡支付',
#     '人工支付充值': '人工支付'
# }
# for old_col, new_col in old_new_columns.items():
#     dfs[2][new_col] = dfs[2][old_col]

# # 刪除舊欄位
# drop_col = ['银联充值', '人工支付充值', 'QQ钱包      充值', '云闪付充值', '支付宝充值']
# dfs[2] = dfs[2].drop(columns = drop_col)

In [145]:
# 1、2月欄位有誤?
set(dfs[10].columns) == set(dfs[9].columns)
# 處理1月的欄位
dfs[10] = dfs[10].drop(columns = ['提现金额'])
dfs[10] = dfs[10].drop(columns = ['活跃用户数'])
# 處理2月的欄位
dfs[9] = dfs[9].drop(columns = ['活跃用户数'])

In [146]:
# 合併所有調整後的df
combined_df = pd.concat(dfs, ignore_index=True, join='outer')

In [147]:
# 刪除含有加總數據的列
drop_index = list(combined_df[combined_df['统计日期'] == '加总数据'].index)
combined_df = combined_df.drop(index=list(drop_index))

In [148]:
# 變更日期datetime格式
combined_df['统计日期'] = pd.to_datetime(combined_df['统计日期'], errors='coerce')

In [149]:
# 刪除一列na
drop_index = list(combined_df[combined_df['统计日期'].isna()].index)
combined_df = combined_df.drop(index=list(drop_index))

In [150]:
# 刪除2023-11-15 到 2023-11-30 無資料
start_date = pd.Timestamp('2023-11-15')
end_date = pd.Timestamp('2023-11-30')
combined_df = combined_df[~combined_df['统计日期'].between(start_date, end_date)]

In [151]:
combined_df_col_cn = combined_df.columns
combined_df_col_cn

Index(['统计日期', '登录用户数', '登录同比', '新增注册用户数', '新增注册同比', '扣量后注册', '留存用户', '留存同比', '昨日安装', '对比昨日', '昨日同比率', '扣量前费用', '扣量后费用', '渠道营收', '提现（元）', '充值人数', '充值人数同比', '新用户充值', '新用户充值人数', '新增会员数', '新增会员数同比', '总充值金额', '人工支付', '支付宝支付', '银行卡支付', '微信支付', '云闪付', 'QQ钱包', '直推充值金额', '（旧版）苹果充值', '（旧版）苹果人均充值', '安卓充值', '安卓人均充值', 'H5充值', 'H5人均充值', '实际拉起订单数', '拉起成功订单数', '成功订单数', '成功率', '游戏人均付费', '游戏营收', '会员总收益', '次日留存', '三日留存', '7日留存', '二次付费率', 'arpu', 'arppu', '备注'], dtype='object')

In [152]:
# 變更欄位名稱
combined_df_col_en = [
'Date',
'Active Users',
'Active Users DoD pct',
'New Registrations',
'New Registrations DoD pct',
'Post-Deduction Registrations',
'Retained Users',
'Retained Users DoD pct',
'Installation yest',
'Installation DoD',
'Installation DoD pct',
'Pre-deduction Cost',
'Post-deduction Cost',
'Channel Revenue',
'Withdrawal Amount',
'Recharge Users',
'Recharge Users DoD pct',
'New User Recharge amount',
'New User Recharge',
'New Membership',
'New Membership DoD',
'Recharge Amount',
'Manual Payment',
'Alipay',
'Bank Card',
'WeChat Pay',
'UnionPay',
'QQ Wallet',
'Direct Referral Recharge Amount',
'iOS Recharge Amount Old Ver',
'iOS AVG Recharge Old Ver',
'Android Recharge Amount',
'Android AVG Recharge',
'H5 Recharge Amount',
'H5 AVG Recharge',
'Triggered Orders',
'Successful Orders',
'Completed Orders',
'Completion Rate',
'AGSPU',
'Game Revenue',
'Membership Revenue',
'1d Retention',
'3d Retention',
'7d Retention',
'Repurchase Rate',
'ARPU',
'ARPPU',
'Remarks'
]

In [153]:
pair = dict(zip(combined_df_col_cn, combined_df_col_en))
pair

{'统计日期': 'Date',
 '登录用户数': 'Active Users',
 '登录同比': 'Active Users DoD pct',
 '新增注册用户数': 'New Registrations',
 '新增注册同比': 'New Registrations DoD pct',
 '扣量后注册': 'Post-Deduction Registrations',
 '留存用户': 'Retained Users',
 '留存同比': 'Retained Users DoD pct',
 '昨日安装': 'Installation yest',
 '对比昨日': 'Installation DoD',
 '昨日同比率': 'Installation DoD pct',
 '扣量前费用': 'Pre-deduction Cost',
 '扣量后费用': 'Post-deduction Cost',
 '渠道营收': 'Channel Revenue',
 '提现（元）': 'Withdrawal Amount',
 '充值人数': 'Recharge Users',
 '充值人数同比': 'Recharge Users DoD pct',
 '新用户充值': 'New User Recharge amount',
 '新用户充值人数': 'New User Recharge',
 '新增会员数': 'New Membership',
 '新增会员数同比': 'New Membership DoD',
 '总充值金额': 'Recharge Amount',
 '人工支付': 'Manual Payment',
 '支付宝支付': 'Alipay',
 '银行卡支付': 'Bank Card',
 '微信支付': 'WeChat Pay',
 '云闪付': 'UnionPay',
 'QQ钱包': 'QQ Wallet',
 '直推充值金额': 'Direct Referral Recharge Amount',
 '（旧版）苹果充值': 'iOS Recharge Amount Old Ver',
 '（旧版）苹果人均充值': 'iOS AVG Recharge Old Ver',
 '安卓充值': 'Android Recharge Amount',


In [154]:
# 輸出欄位對照表
op_df_trans = pd.DataFrame(list(pair.items()), columns=['中文欄位名', '英文欄位名'])
op_df_trans.to_excel('op_df_trans.xlsx', index=False)

In [155]:
combined_df.rename(columns=pair, inplace=True)

In [156]:
# # 刪除重複欄位
# combined_df = combined_df.drop(columns=['Newly Registered Users DoD(%)2'])

In [157]:
# combined_df.to_excel("sp001_tb_op.xlsx", index=False)

In [160]:
combined_df.to_csv("sp001_tb_op.csv", index=False, encoding='utf-8')

#### 3. Top-up dataset

In [3]:
#　7/11-11/15

In [45]:
end_day = '2023-11-14'
start_day = '2023-07-11'
 
date_range = pd.date_range(start_day, end_day, freq = "1D")
date_range_str = [date_range.strftime('%Y-%m-%d') for date_range in date_range] # list 

In [46]:
files_path_list = []

for i in date_range_str:
    files_path = glob(".\\all_channel_rawdata\\" + 
                  i + 
                  "\\top_up\\charge_*.xlsx")
    files_path_list.extend(files_path)

In [47]:
files_path_list

['.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689141827052.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689141849256.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689141956193.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689141993171.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689142009761.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689142110626.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689142153966.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689142169643.xlsx',
 '.\\all_channel_rawdata\\2023-07-11\\top_up\\charge_1689142188672.xlsx',
 '.\\all_channel_rawdata\\2023-07-12\\top_up\\charge_1689230542083.xlsx',
 '.\\all_channel_rawdata\\2023-07-12\\top_up\\charge_1689230561042.xlsx',
 '.\\all_channel_rawdata\\2023-07-12\\top_up\\charge_1689230585902.xlsx',
 '.\\all_channel_rawdata\\2023-07-12\\top_up\\charge_1689230605414.xlsx',
 '.\\all_channel_rawdata\\2023-07-12\\

In [ ]:
# 合併檔案
all_top_up = pd.DataFrame()
for file in files_path_list:
    df = pd.read_excel(file, engine='openpyxl')
    all_top_up = pd.concat([all_top_up, df],  ignore_index=True)

In [65]:
for index, col_name in enumerate(all_top_up.columns):
    print(f"no.{index}, col_name:{col_name}")

no.0, col_name:订单发起时间
no.1, col_name:订单完成时间
no.2, col_name:用户id
no.3, col_name:用户昵称
no.4, col_name:手机
no.5, col_name:订单号
no.6, col_name:通道订单号
no.7, col_name:充值用途
no.8, col_name:充值通道
no.9, col_name:支付类型
no.10, col_name:订单名称
no.11, col_name:充值金额
no.12, col_name:到账金额
no.13, col_name:订单状态
no.14, col_name:app应用类型
no.15, col_name:app渠道名称
no.16, col_name:支付的ip
no.17, col_name:地区
no.18, col_name:设备ID


In [66]:
t = all_top_up.iloc[:, [0, 1, 2, 5, 7, 8, 9, 10, 11, 12, 14, 16]]

In [67]:
t['订单发起时间'] = pd.to_datetime(t['订单发起时间'])
t['订单完成时间'] = pd.to_datetime(t['订单完成时间'])
t['充值金额'] = t['充值金额'].astype(int)

t = t.reset_index(drop=True)
t.drop_duplicates(inplace=True)

C:\Users\Rex\AppData\Local\Temp\ipykernel_16044\3147785438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['订单发起时间'] = pd.to_datetime(t['订单发起时间'])
C:\Users\Rex\AppData\Local\Temp\ipykernel_16044\3147785438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['订单完成时间'] = pd.to_datetime(t['订单完成时间'])
C:\Users\Rex\AppData\Local\Temp\ipykernel_16044\3147785438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [68]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256814 entries, 0 to 277278
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   订单发起时间   256814 non-null  datetime64[ns]
 1   订单完成时间   256814 non-null  datetime64[ns]
 2   用户id     256814 non-null  int64         
 3   订单号      256810 non-null  object        
 4   充值用途     256813 non-null  object        
 5   充值通道     252297 non-null  object        
 6   支付类型     256814 non-null  object        
 7   订单名称     256813 non-null  object        
 8   充值金额     256814 non-null  int32         
 9   到账金额     256814 non-null  float64       
 10  app应用类型  256810 non-null  object        
 11  支付的ip    252262 non-null  object        
dtypes: datetime64[ns](2), float64(1), int32(1), int64(1), object(7)
memory usage: 24.5+ MB


In [69]:
# 欄位轉換
top_up_col_cn = t.columns
top_up_col_en = [
    'Order Time', 'Order Completion Time', 'User ID', 'Order Number', 
    'Purpose', 'Payment Gateway', 'Payment Type', 'Order Type', 
    'Recharge Amount', 'Amount Received', 'Device Type', 'IP']
pair = dict(zip(top_up_col_cn, top_up_col_en))
t.rename(columns= pair, inplace = True)

In [70]:
# 輸出欄位對照表
top_up_df_trans = pd.DataFrame(list(pair.items()), columns=['中文欄位名', '英文欄位名'])
top_up_df_trans.to_excel('top_up_df_trans.xlsx', index=False)

In [71]:
# Payment Type轉英文
Payment_value_cn = [
    '支付寶支付',
    '銀行卡支付',
    'QQ錢包',
    '微信支付',
    '雲閃付',
    '人工支付',
    '波波支付',
    '人工充值'
]

# 英文列表
Payment_value_en = [
    'Alipay',
    'Bank Card',
    'QQ Wallet',
    'WeChat Pay',
    'UnionPay',
    'Manual',
    'Bobo Pay',
    'Manual'
]
Payment_replace_dict = dict(zip(t['Payment Type'].unique(), Payment_value_en))
t['Payment Type'] = t['Payment Type'].replace(Payment_replace_dict)

In [72]:
# Order Type col processing
t['Order Type'].unique()

array(['常规充值', '人工充值', nan, 'QQ钱包', '支付宝支付', '微信支付', '云闪付', '银行卡支付',
       'VIP会员月卡', 'VIP会员季度卡', '波波支付', '现金支付'], dtype=object)

In [73]:
order_type_value_cn = t['Order Type'].unique().tolist()
order_type_value_en = ['Regular Recharge', 
                'Manual', 
                float('nan'), 
                'Regular Recharge', 
                'Regular Recharge', 
                'Regular Recharge', 
                'Regular Recharge', 
                'Regular Recharge',
                'VIP Monthly', 
                'VIP Quarterly', 
                'Regular Recharge', 
                'Cash']

In [74]:
order_type_value_dict = dict(zip(order_type_value_cn, order_type_value_en))
t['Order Type'] = t['Order Type'].replace(order_type_value_dict)

In [75]:
# 有一筆nan，按Payment Type = Manual填補成Manual
print(t[t['Order Type'].isna()])

               Order Time Order Completion Time   User ID Order Number Purpose Payment Gateway Payment Type Order Type  Recharge Amount  Amount Received Device Type   IP
81994 2023-08-29 13:17:12   2023-08-29 13:17:12  39922030          NaN     NaN             NaN       Manual        NaN             3000           3000.0         NaN  NaN


In [76]:
t['Order Type'] = t['Order Type'].fillna('Manual')

In [77]:
# 填補成功
print(t[t['Order Type'].isna()])

Empty DataFrame
Columns: [Order Time, Order Completion Time, User ID, Order Number, Purpose, Payment Gateway, Payment Type, Order Type, Recharge Amount, Amount Received, Device Type, IP]
Index: []


In [78]:
# plt.figure(figsize=(10, 6))
# t['Order Type'].value_counts().plot(kind='bar')
# plt.title('distri')
# plt.ylabel('count')
# plt.xlabel('Order Type')
# plt.xticks(rotation=45)
# plt.show()
# print(t['Order Type'].value_counts())

In [79]:
# purpose col processing

# 購買會員 = Upgrade
purpose_col_cn = t['Purpose'].unique()
purpose_col_en = ['Recharge', 
                  float('nan'), 
                  'Upgrade']
purpose_col_dict = dict(zip(purpose_col_cn, purpose_col_en))
purpose_col_dict

t['Purpose'] = t['Purpose'].replace(purpose_col_dict)

print(t['Purpose'].value_counts())

Purpose
Recharge    256039
Upgrade        774
Name: count, dtype: int64


In [80]:
gateway_col_cn = t['Payment Gateway'].unique()

In [81]:
gateway_col_cn

array(['聚合支付', '利达四方', nan, 'BOBI支付'], dtype=object)

In [82]:
t[ t['Payment Gateway'].isna()].shape[0]

4517

In [83]:
gateway_col_en = ['Aggregated Payment', 'LianLian Pay', 'NA', 'BOBI Payment']

In [84]:
payment_gateway_col_dict = dict(zip(gateway_col_cn, gateway_col_en))
payment_gateway_col_dict

{'聚合支付': 'Aggregated Payment',
 '利达四方': 'LianLian Pay',
 nan: 'NA',
 'BOBI支付': 'BOBI Payment'}

In [85]:
# Payment Gateway col processing
t['Payment Gateway'] = t['Payment Gateway'].replace(payment_gateway_col_dict)

In [86]:
t[ t['Payment Gateway'].isna()].shape[0]

0

In [87]:
# device type 轉換大小寫
t['Device Type'] = t['Device Type'].replace('IOS', 'iOS')
t['Device Type'] = t['Device Type'].replace('ANDROID', 'Android')

In [88]:
t['Device Type'].fillna('no record', inplace=True)
#### missing value processing

C:\Users\Rex\AppData\Local\Temp\ipykernel_16044\1510935752.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  t['Device Type'].fillna('no record', inplace=True)


In [89]:
# missing value processing

In [90]:
print(t.isna().sum())

Order Time                  0
Order Completion Time       0
User ID                     0
Order Number                4
Purpose                     1
Payment Gateway             0
Payment Type                0
Order Type                  0
Recharge Amount             0
Amount Received             0
Device Type                 0
IP                       4552
dtype: int64


In [91]:
#　ip na過多
naip = t[ t['IP'].isna()]

In [ ]:
# 剛好Device Type幾乎是H5
naip

In [93]:
# 也有iOS, Android, 共39筆
not_h5 = naip[naip['Device Type'] != 'H5']
not_h5.shape[0]

39

In [94]:
# Order Number, Purpose, Device Type的missing value

na_row = t[t['Order Number'].isnull() | t['Purpose'].isnull()]
na_row

,Order Time,Order Completion Time,User ID,Order Number,Purpose,Payment Gateway,Payment Type,Order Type,Recharge Amount,Amount Received,Device Type,IP
81994,2023-08-29 13:17:12,2023-08-29 13:17:12,39922030,NaN,NaN,NA,Manual,Manual,3000,3000.0,no record,NaN
86254,2023-08-31 01:10:55,2023-08-31 01:10:55,39777257,NaN,Recharge,NA,Manual,Manual,2000,2000.0,no record,NaN
112574,2023-09-12 22:37:10,2023-09-12 22:40:56,41033000,NaN,Recharge,LianLian Pay,Bank Card,Regular Recharge,1000,1000.0,Android,14.150.131.170
112575,2023-09-12 22:35:49,2023-09-12 22:38:54,41031696,NaN,Recharge,LianLian Pay,Bank Card,Regular Recharge,2000,2000.0,Android,14.150.131.170


In [95]:
t['Purpose'].fillna('no record', inplace = True)

C:\Users\Rex\AppData\Local\Temp\ipykernel_16044\3104758203.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  t['Purpose'].fillna('no record', inplace = True)


In [96]:
# t.to_excel("sp001_tb_recharge.xlsx", index=False)

In [161]:
t.to_csv("sp001_tb_recharge.csv", index=False, encoding='utf-8')

#### 4. Membership_Sign_Up_dataset

In [166]:
file_list = glob(".\\data_source\\membership_purchase\\charge_*.xlsx")

In [167]:
file_list

['.\\data_source\\membership_purchase\\charge_1694152223894.xlsx',
 '.\\data_source\\membership_purchase\\charge_1694152261327.xlsx',
 '.\\data_source\\membership_purchase\\charge_1694152272822.xlsx',
 '.\\data_source\\membership_purchase\\charge_1694152401634.xlsx',
 '.\\data_source\\membership_purchase\\charge_1694152412728.xlsx',
 '.\\data_source\\membership_purchase\\charge_1695437923429.xlsx',
 '.\\data_source\\membership_purchase\\charge_1695437935386.xlsx',
 '.\\data_source\\membership_purchase\\charge_1695438319166.xlsx']

In [170]:
combined_df = pd.DataFrame()

In [172]:
all_df = []
for file in file_list:
    df = pd.ExcelFile(file)
    for sheet in df.sheet_names:
        df_sheet = pd.read_excel(df, sheet_name=sheet)
        all_df.append(df_sheet)
        

c:\Users\Rex\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Rex\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Rex\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Rex\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Rex\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no defaul

In [173]:
combined_df = pd.concat(all_df, ignore_index=True)

In [ ]:
combined_df 

In [246]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 0 to 1277
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Order Time             1278 non-null   datetime64[ns]
 1   Order Completion Time  1278 non-null   datetime64[ns]
 2   User ID                1278 non-null   object        
 3   Order Number           1278 non-null   object        
 4   Channel Order Number   1278 non-null   int64         
 5   Purpose                1278 non-null   object        
 6   Payment Gateway        1278 non-null   object        
 7   Payment Type           1278 non-null   object        
 8   Membership Level       1278 non-null   object        
 9   Recharge Amount        1278 non-null   float64       
 10  Amount Received        1278 non-null   float64       
 11  Device Type            1278 non-null   object        
 12  IP                     1278 non-null   object        
 13  Dev

In [ ]:
combined_df.drop(columns=['用户昵称', '手机', '订单状态', 'app渠道名称', '地区'], inplace=True)

In [212]:
membership_col_cn = combined_df.columns.tolist()

In [213]:
membership_col_en = [
    "Order Time",
    "Order Completion Time",
    "User ID",
    "Order Number",
    "Channel Order Number",
    "Purpose",
    "Payment Gateway",
    "Payment Type",
    "Membership Level",
    "Recharge Amount",
    "Amount Received",
    "Device Type",
    "IP",
    "Device ID",
    "Order Date"
]

In [214]:
pair = dict(zip(membership_col_cn, membership_col_en))
combined_df.rename(columns= pair, inplace = True)

In [223]:
# 輸出欄位對照表
membership_df_trans = pd.DataFrame(pair.items(), columns=['中文欄位名', '英文欄位名'])
membership_df_trans.to_excel('membership_df_trans.xlsx', index=False)

In [225]:
# 修改時間格式
combined_df[['Order Time', 'Order Completion Time']] = combined_df[['Order Time', 'Order Completion Time']].apply(pd.to_datetime)

In [ ]:
# 新增訂單日期欄位 
combined_df['Order Date'] = combined_df['Order Time'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [245]:
# User ID fromat int -> str
combined_df['User ID'] = combined_df['User ID'].apply(lambda x: str(x))

In [247]:
# # Channel Order Number fromat int -> str
combined_df['Channel Order Number'] = combined_df['Channel Order Number'].apply(lambda x: str(x))

In [ ]:
# 提換為英文值

In [256]:
# replace purpose value
purpose_cn_value = combined_df['Purpose'].unique().tolist()
purpose_en_value = ['Upgrade']
purpose_pair = dict(zip(purpose_cn_value, purpose_en_value))
combined_df['Purpose'] = combined_df['Purpose'].replace(purpose_pair)

In [265]:
# replace gateway value
gateway_cn_value = combined_df['Payment Gateway'].unique().tolist()
gateway_en_value = ['Aggregated Payment']
gateway_pair = dict(zip(gateway_cn_value, gateway_en_value))
combined_df['Payment Gateway'] = combined_df['Payment Gateway'].replace(gateway_pair)

In [266]:
# replace payment value
payment_cn_value = combined_df['Payment Type'].unique().tolist()
payment_en_value = ['WeChat Pay', 'Alipay', 'QQ Wallet', 'UnionPay']
payment_pair = dict(zip(payment_cn_value, payment_en_value))
combined_df['Payment Type'] = combined_df['Payment Type'].replace(payment_pair)

In [270]:
# replace level value
level_cn_value = combined_df['Membership Level'].unique().tolist()
level_en_value = ['VIP Monthly', 'VIP Quarterly']
level_pair = dict(zip(level_cn_value, level_en_value))
combined_df['Membership Level'] = combined_df['Membership Level'].replace(level_pair)

In [271]:
# Device Type value
combined_df['Device Type'] = combined_df['Device Type'].replace('IOS', 'iOS')
combined_df['Device Type'] = combined_df['Device Type'].replace('ANDROID', 'Android')

In [204]:
# 會員數
len(combined_df['用户id'].unique().tolist())

1235

In [196]:
combined_df['订单发起日期'].min()

'2023-08-25'

In [197]:
combined_df['订单发起日期'].max()

'2023-09-30'

In [272]:
# combined_df.to_excel("Project_dataset_membership.xlsx", index=False)
combined_df.to_csv("sp001_tb_membership.csv", index=False, encoding='utf-8')

#### 5. withdrawal_dataset

In [ ]:
# 8月25-31
# 9月完整
# 10月1-5

#### 6. new_user_recharge

In [ ]:
# 06/19-11/12

In [77]:
new_user_recharge_path = glob("D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\*.xlsx")
new_user_recharge_path = sorted(new_user_recharge_path, key=lambda x: int(x.split('\\')[-1].split('_')[0][1:]), reverse=False)
new_user_recharge_path

['D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m6_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m7_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m8_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m9_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m10_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m11_new_user_recharge.xlsx']

In [78]:
# v1 sheet

new_user_recharge_path_list = []
for i in new_user_recharge_path[0:4]:
    new_user_recharge_path_list.append(i)

new_user_recharge_path_list



['D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m6_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m7_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m8_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m9_new_user_recharge.xlsx']

In [79]:
new_user_recharge_file = pd.DataFrame()
for file in new_user_recharge_path_list:
    sheet_names = pd.ExcelFile(file, engine='openpyxl').sheet_names
    for sheet_name in sheet_names:
        each_file = pd.read_excel(file, sheet_name=sheet_name, engine='openpyxl')
        new_user_recharge_file = pd.concat([new_user_recharge_file, each_file], ignore_index=True)

In [80]:
new_user_recharge_v1 = new_user_recharge_file

In [91]:
new_user_recharge_v1 = new_user_recharge_v1.drop(columns=["Unnamed: 0", "提现", "购买会员"])

In [81]:
new_user_recharge_v1.rename(columns={'总充值（元）':'总充值'},inplace=True)
new_user_recharge_v1.rename(columns={'新用户充值金额':'新用户充值'},inplace=True)

In [82]:
# v2
new_user_recharge_path_list = []
for i in new_user_recharge_path[4:]:
    new_user_recharge_path_list.append(i)

new_user_recharge_path_list


['D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m10_new_user_recharge.xlsx',
 'D:\\Rex\\Desktop\\DS_workspace\\sp001_workdata_analysis\\ETL_workspace\\data_source\\new_user_recharge_analysis\\m11_new_user_recharge.xlsx']

In [84]:
new_user_recharge_file = pd.DataFrame()
for i in new_user_recharge_path_list:
    sheet_names = pd.ExcelFile(i, engine='openpyxl').sheet_names
    for sheet_name in sheet_names:
        each_file = pd.read_excel(i, sheet_name=sheet_name, engine='openpyxl')
        new_user_recharge_file = pd.concat([new_user_recharge_file, each_file], ignore_index=True)


In [85]:
new_user_recharge_v2 = new_user_recharge_file
# new_user_recharge_v2.sort_values(by=['日期'], ascending=True)

In [87]:
new_user_recharge_v2.rename(columns={'新用户充值(元)':'新用户充值'},inplace=True)

In [92]:
new_user_recharge_v2 = new_user_recharge_v2.drop(columns=['Unnamed: 0'])

In [93]:
new_user_recharge_col_cn_v1 = new_user_recharge_v1.columns
new_user_recharge_col_cn_v2 = new_user_recharge_v2.columns
new_user_recharge_col_cn_v1 == new_user_recharge_col_cn_v2

array([ True,  True,  True,  True,  True,  True,  True,  True])

In [94]:
new_user_recharge = pd.concat([new_user_recharge_v1, new_user_recharge_v2], ignore_index=True)

In [32]:
new_user_recharge

,日期,渠道名称,日活,新增注册,充值人数,新增会员,总充值,新用户充值,提现,购买会员
0,2023-06-19,7021渠道,8153,724,33,0,10050,100,0.0,0.0
1,2023-06-19,7022渠道,30846,2255,181,4,67850,0,0.0,250.0
2,2023-06-19,7023渠道,40577,7619,102,5,17600,650,0.0,250.0
3,2023-06-19,7024渠道,4267,804,1,0,50,0,0.0,0.0
4,2023-06-19,7025渠道,786,5,4,0,800,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
35549,2023-11-12,7298渠道,392,145,0,0,0,0,NaN,NaN
35550,2023-11-12,7299渠道,74,0,0,0,0,0,NaN,NaN
35551,2023-11-12,7300渠道,0,0,0,0,0,0,NaN,NaN
35552,2023-11-12,7301渠道,1712,1209,0,0,0,0,NaN,NaN


In [96]:
new_user_recharge_col_cn = new_user_recharge.columns
new_user_recharge_col_cn

Index(['日期', '渠道名称', '日活', '新增注册', '充值人数', '新增会员', '总充值', '新用户充值'], dtype='object')

In [101]:
new_user_recharge_col_en = [
    'Date',
    'Channel Name',
    'Active Users',
    'New Registrations',
    'Recharge Users',
    'New Membership',
    'Recharge Amount',
    'New User Recharge Amount'
]
new_user_recharge_col_en

['Date',
 'Channel Name',
 'Active Users',
 'New Registrations',
 'Recharge Users',
 'New Membership',
 'Recharge Amount',
 'New User Recharge Amount']

In [102]:
new_user_recharge_col_replace_dict = dict(zip(new_user_recharge_col_cn , new_user_recharge_col_en))
new_user_recharge_col_replace_dict

{'日期': 'Date',
 '渠道名称': 'Channel Name',
 '日活': 'Active Users',
 '新增注册': 'New Registrations',
 '充值人数': 'Recharge Users',
 '新增会员': 'New Membership',
 '总充值': 'Recharge Amount',
 '新用户充值': 'New User Recharge Amount'}

In [103]:
new_user_recharge_df_trans = pd.DataFrame(new_user_recharge_col_replace_dict.items(), columns=['中文欄位名', '英文欄位名'])
new_user_recharge_df_trans.to_excel('new_user_recharge_df_trans.xlsx', index=False)

In [104]:
new_user_recharge.rename(columns=new_user_recharge_col_replace_dict, inplace=True)

In [ ]:
new_user_recharge

In [105]:
new_user_recharge['Channel Name'] = new_user_recharge['Channel Name'].str.replace('渠道', '')

In [106]:
new_user_recharge.to_csv("sp001_tb_new_user_recharge.csv", index=False, encoding='utf-8')